In [1]:
import os 
import re
import sys
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf

In [ ]:
#Reading the tatoeba file

In [ ]:
# lns=[]
# with open("sentences.csv",'r',encoding='utf-8') as f:
#     while True:
#         line = f.readline()
#         if not line:
#             break
#         if not re.search('\seng\s',line,re.I):
#             continue
#         try:
#             line = line.split('\t')
#             line = line[2]
#             lns.append(line)
#         except: pass

In [ ]:
# len(lns)

In [ ]:
# lns[:10]

In [ ]:
lns = sorted(lns, key=len, reverse=False)

In [ ]:
lns[:10]

In [ ]:
with open('english_sentences_tatoeba.txt','w',encoding='utf-8') as f:
    for line in lns:
        f.write('%s'%line)

In [2]:
with open('english_sentences_tatoeba.txt','r',encoding='utf-8') as f:
    lns = f.readlines()
print(len(lns),lns[:6])

1275343 ['Hi\n', 'So?\n', 'Ow!\n', 'Ok.\n', 'Oi!\n', 'OK.\n']


In [ ]:
lns = [l.replace('\n','') for l in lns]

In [ ]:
#using nltk to tokenize sentences furthur

In [ ]:
from nltk import sent_tokenize, word_tokenize

In [ ]:
lns_n=[]
for l in lns:
    l = sent_tokenize(l)
    lns_n.extend(l)
lns_n = sorted(lns_n , key=len , reverse=False)
print(len(lns))
print("new_length::",len(lns_n))
    

In [ ]:
#creating boundaries 
a = (print(l) for l in lns_n if ' and ' in l)

In [ ]:
while True:
    print(next(a))
    

In [ ]:
iter(a)

In [ ]:
from nltk import pos_tag

In [ ]:
lns_pos = [pos_tag(l.split(' ')) for l in lns_n[10000:11800]]

In [ ]:
lns_pos=[]
for l in lns_n[10000:12000]:
    l = word_tokenize(l)
    if len(l)<4:
        continue
    l = pos_tag(l)
    lns_pos.append(l)

In [ ]:
len(lns_pos)

In [ ]:
lns_pos

In [3]:
def generate_data(lines, max_sents_per_example=6, n_examples=1000):
    """
        Generates training data for deepsegment from list of sentences.
        Parameters:
        lines (list): Base sentences for data generation.
        max_sents_per_example (int): Maximum number of sentences to be combined to form a single paragraph.
        
        n_examples (int): Number of training examples to be generated.
        
        Returns:
        list, list: Training data and corresponding labels in BIOU format.
    """
    x, y = [], []
    
    for current_i in tqdm(range(n_examples)):
        x.append([])
        y.append([])

        chosen_lines = []
        for _ in range(random.randint(1, max_sents_per_example)):
            chosen_lines.append(random.choice(lines))
        
        chosen_lines = [bad_sentence_generator(line, remove_punctuation=random.randint(0, 3)) for line in chosen_lines]
        
        for line in chosen_lines:
            words = line.strip().split()
            for word_i, word in enumerate(words):
                x[-1].append(word)
                label = 'O'
                if word_i == 0:
                    label = 'B-sent'
                y[-1].append(label)
    
    return x, y


def bad_sentence_generator(sent, remove_punctuation = None):
    """
        Returns sentence with completely/ partially removed punctuation.
        Parameters:
        sent (str): Sentence on which the punctuation removal operation is performed.
        
        remove_punctuation (int): removing punctuation completely if remove_punctuation ==0 or ==1, removing punctuation till a randomly selected point if remove_punctuation ==2
        Returns:
        str: Sentence with modified punctuation
    """

    if not remove_punctuation:
        remove_punctuation = random.randint(0, 3)

    break_point = random.randint(1, len(sent)-2)
    lower_case = random.randint(0, 2)

    if remove_punctuation <= 1:
        # removing punctuation completely if remove_punctuation ==0 or ==1
        sent = re.sub('['+string.punctuation+']', '', sent)
    
    elif remove_punctuation == 2:
        # removing punctuation till a randomly selected point if remove_punctuation ==2
        if random.randint(0,1) == 0:
            sent = re.sub('['+string.punctuation+']', '', sent[:break_point]) + sent[break_point:]
        # removing punctuation after a randomly selected point if remove_punctuation ==2        
        else:
            sent = sent[:break_point] + re.sub('['+string.punctuation+']', '', sent[break_point:])    
    
    if lower_case <= 1:
        # lower casing sentence 
        sent = sent.lower()
    
    return sent


In [4]:
from tqdm import tqdm
import random
import string
string.punctuation
lns[:10]

['Hi\n',
 'So?\n',
 'Ow!\n',
 'Ok.\n',
 'Oi!\n',
 'OK.\n',
 'Nu?\n',
 'No?\n',
 'No.\n',
 'No!\n']

In [5]:
x, y = generate_data(lines=lns[10000:], max_sents_per_example=6, n_examples=10000)
x_, y_ = generate_data(lines=lns[:10000], max_sents_per_example=6, n_examples=1000)

100%|███████████████████████████████████| 1000/1000 [00:00<00:00, 25657.32it/s]


In [6]:
# x[:1],y[:1]

In [7]:
#download the glove word embeddings

In [8]:
os.listdir('glove')

['glove.6B.100d.txt',
 'glove.6B.200d.txt',
 'glove.6B.300d.txt',
 'glove.6B.50d.txt']

In [9]:
def load_vocab(filename):
    """Loads vocab from a file
    Args:
        filename: (string) the format of the file must be one word per line.
    Returns:
        d: dict[word] = index
    """
    try:
        d = dict()
        with open(filename) as f:
            for idx, word in enumerate(f):
                word = word.strip()
                d[word] = idx

    except IOError:
        raise MyIOError(filename)
    return d


In [22]:
#glove embedding
glove_path = "glove/glove.6B.100d.txt"

In [30]:
def get_glove_vocab(path):
    print('Loading word Vectors....')
    word2vec = {}
    with open(path,'r',encoding='utf-8') as f:
      for line in f:
        values = line.split()
        word = values[0]
        vec = np.array(values[1:],dtype='float32')
        word2vec[word] = vec
    print('Found %s word vectors.' %len(word2vec))
    return word2vec

word2vec = get_glove_vocab(glove_path) 

Loading word Vectors....
Found 400000 word vectors.


In [19]:
embedding_dict = load_vocab("glove/glove.6B.100d.txt")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2776: character maps to <undefined>

In [ ]:
# def train(x, y, vx, vy, epochs, batch_size, save_folder, glove_path):
#     """
#         Trains a deepsegment model.
#         Parameters:
#         x (list): x generated from generate_data
#         y (list): y generated from generate_data
#         vx (list): x generated from generate_data
#         vy (list): y generated from generate_data
#         epochs (int): Max number of epochs.
        
#         batch_size (int): batch_size
#         save_folder (str): path for the directory where checkpoints should be saved.
#         glove_path (str): path to 100d word vectors.
        
#     """

#     embeddings = load_vocab(glove_path)
    
#     checkpoint_path = os.path.join(save_folder, 'checkpoint')
#     final_weights_path = os.path.join(save_folder, 'final_weights')
#     params_path = os.path.join(save_folder, 'params')
#     utils_path = os.path.join(save_folder, 'utils')    

#     checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True, mode='max', monitor='f1')
#     earlystop = EarlyStopping(patience=3, monitor='f1', mode='max')

#     model = seqtag_keras.Sequence(embeddings=embeddings)
    
#     model.fit(x, y, x_valid=vx, y_valid=vy, epochs=epochs, batch_size=batch_size, callbacks=[checkpoint, earlystop])

#     model.save(final_weights_path, params_path, utils_path)


# lang_code_mapping = {
#     'english': 'en',
#     'french': 'fr',
#     'italian': 'it'
# }


In [10]:
#build model
import tensorflow as tf

In [11]:
tf.reset_default_graph()

In [12]:
# def intialize_placeholders():
word_ids = tf.placeholder(tf.int32, shape=[None,None], name="word_id_placeholder")
sequence_length = tf.placeholder(tf.int32, shape=[None,], name="sequence_length_placholder")
label = tf.placeholder(tf.int32,shape=[None,None], name="labels")
#         dropout = tf.placeholder(dtype=tf.float32, shape=[],
#                         name="dropout")
lr = tf.placeholder(dtype=tf.float32, shape=[],
                name="lr")

In [15]:
nwords=20000,
embedding_size = 100
hidden_dim = 100

In [18]:
def word_embedding(embedding_dict=None,nwords=10000,embedding_size=100):
    with tf.variable_scope("word_embedding"):
        if embedding_dict is None:
            word_embeddings_ = tf.get_variable(name="word_embeddings_",
                                              dtype=tf.float32,
                                              shape=[nwords,embedding_size])
        else:
            word_embeddings_ = tf.variable(embedding_dict,
                                           name="word_embeddings_",
                                           dtype=tf.float32,
                                           shape=[nwords,embedding_size],
                                           trainable=True
                                           )
            
        word_embedding = tf.nn.embedding_lookup(word_embeddings_,"words_ids",name="word_embeddings")

    return word_embedding
        
        

            

TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [ ]:
# def logits_op(hidden_dim):
with tf.variable_scope("bidirectional_lstm"):
    for_cell = tf.contrib.rnn.LSTMCell(hidden_dim)
    bac_cell = tf.contrib.rnn.LSTMCell(hidden_dim)
    (for_out, bac_out), _ = tf.nn.bidirectional_dynamic_rnn(for_cell, bac_cell,
                                                            word_embeddings,
                                                            sequence_length=sequence_length,
                                                            dtype=tf.float32)
    #shape >> [bs, sequence_length, 2EMD]
    output = tf.concat([for_out, bac_out], axis=-1)

#         output = tf.nn.dropout(output,)

with tf.variable_scope("w_b"):
    W = tf.get_variable(name="W",dtype=tf.float32,
                        shape = [2*hidden_dim,ntags])
    b = tf.get_variable(name="b", dtype=tf.float32,
                        shape = [ntags],
                        initializer = tf.zeros_initializer())


    nsteps = tf.shape(output)[1]
    output = tf.reshape(output, [-1,2*hidden_dim])

    pred = tf.matmul(output,W) + b
    logits = tf.reshape(pred,[-1,nsteps,ntags])



        
    

In [ ]:
# def pred_op():
label_pred = tf.cast(tf.argmax(logits, axis=-1),tf.int32)

In [ ]:
# def loss_op():
losses = tf.nn.sparse_softmax_crossentropy_with_logits(logits=logits,labels=labels)
mask = tf.sequence_mask(self.sequence_lengths)
losses = tf.boolean_mask(losses, mask)
loss = tf.reduce_mean(losses)
optimizer = tf.train.AdamOptimizer(lr)
training_op = optimizer.minimize(loss)

In [1]:
def initialize_session():
    sess = tf.Session()
    init = tf.global_variable_initializers()
    sess.run(init)
    saver = tf.train.Saver()
    return sess, saver

In [ ]:
def save_session():
    if not os.path.exist('model_folder'):
        os.makedirs('model_folder')
    saver.save(sess, 'model_folder')

def close_sess():
    sess.close()

In [ ]:
def minibatches(data,batch_size):
    x_batch, y_batch = [], []
    for (x,y) in data:
        if len(x_batch) == batch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []
        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]
        
    if len(x_batch) != 0:
        yield x_batch, y_batch

In [ ]:
def get_chunk_type(tok, idx_to_tag):
    """
    Args:
        tok: id of token, ex 4
        idx_to_tag: dictionary {4: "B-PER", ...}
    Returns:
        tuple: "B", "PER"
    """
    tag_name = idx_to_tag[tok]
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type


def get_chunks(seq, tags):
    """Given a sequence of tags, group entities and their position
    Args:
        seq: [4, 4, 0, 0, ...] sequence of labels
        tags: dict["O"] = 4
    Returns:
        list of (chunk_type, chunk_start, chunk_end)
    Example:
        seq = [4, 5, 0, 3]
        tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
        result = [("PER", 0, 2), ("LOC", 3, 4)]
    """
    if NONE in tags:
        default = tags[NONE]
    else:
        default = None
        
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)

    return chunks

In [ ]:
def predict_batch(words):
    fd, sequence_length = get_feed_dict(words) #dropout=1.0)
    labels_pred = sess.run(label_pred, feed_dict=fd)
    return labels_pred, sequence_length

In [ ]:
def _pad_sequences(sequences, pad_tok, max_length):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
    Returns:
        a list of list where each sublist has same length
    """
    sequence_padded, sequence_length = [], []

    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok]*max(max_length - len(seq), 0)
        sequence_padded +=  [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length


def pad_sequences(sequences, pad_tok, nlevels=1):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
        nlevels: "depth" of padding, for the case where we have characters ids
    Returns:
        a list of list where each sublist has same length
    """
    if nlevels == 1:
        max_length = max(map(lambda x : len(x), sequences))
        sequence_padded, sequence_length = _pad_sequences(sequences,
                                            pad_tok, max_length)

    elif nlevels == 2:
        max_length_word = max([max(map(lambda x: len(x), seq))
                               for seq in sequences])
        sequence_padded, sequence_length = [], []
        for seq in sequences:
            # all words are same length now
            sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
            sequence_padded += [sp]
            sequence_length += [sl]

        max_length_sentence = max(map(lambda x : len(x), sequences))
        sequence_padded, _ = _pad_sequences(sequence_padded,
                [pad_tok]*max_length_word, max_length_sentence)
        sequence_length, _ = _pad_sequences(sequence_length, 0,
                max_length_sentence)

    return sequence_padded, sequence_length

In [ ]:
def get_feed_dict(words, labels=None, lr=None, dropout=None):
        """Given some data, pad it and build a feed dictionary
        Args:
            words: list of sentences. A sentence is a list of ids of a list of
                words. A word is a list of ids
            labels: list of ids
            lr: (float) learning rate
            dropout: (float) keep prob
        Returns:
            dict {placeholder: value}
        """
        # perform padding of the given data
#         if self.config.use_chars:
#             char_ids, word_ids = zip(*words)
#             word_ids, sequence_lengths = pad_sequences(word_ids, 0)
#             char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
#                 nlevels=2)
#         else:
        word_ids, sequence_lengths = pad_sequences(words, 0)

        # build feed dictionary
        feed = {
            "word_id_placeholder": word_ids,
            "sequence_length_placholder": sequence_lengths
        }

#         if self.config.use_chars:
#             feed[self.char_ids] = char_ids
#             feed[self.word_lengths] = word_lengths
        if labels is not None:
            labels, _ = pad_sequences(labels, 0)
            feed["labels"] = labels

        if lr is not None:
            feed["lr"] = lr

#         if dropout is not None:
#             feed["dropout"] = dropout

        return feed, sequence_lengths

In [ ]:
def run_epoch(train, dev, epoch):
    for i ,(words, labels) in enumerate(minibatches(train,batch_size)):
        fd, _ = get_feed_dict(words, labels, lr)#, dropout)
        _, train_los = sess.run(training_op, loss, feed_dict = fd)
        print("training_loss::::",train_los)
        
        metrics = evaluation(dev)
        msg = " - ".join(["{} {:04.2f}".format(k, v)
                for k, v in metrics.items()])
        print(msg)
        print(metrics['f1'])
        

def evaluation(dev):
    accs = []
    correct_prediction, total_correct, total_predictions = 0., 0., 0.
    for words, labels in minibatches(dev, batch_size):
        label_preds, sequence_length = predict_batch(words)
        
        for lab, lab_pred, length in zip(labels, label_preds, sequence_length):
            lab = lab[:length]
            lab_pred = lab_pred[:length]
            
            accs += [a==b for (a,b) in zip(lab,lab_pred)]
            
            lab_chunks = set(get_chunks(lab, vocab_tags))
            lab_pred_chunks = set(get_chunks(lab_pred, vocab_tags))
            
            correct_preds += len(lab_chunks & lab_pred_chunks)
            total_preds   += len(lab_pred_chunks)
            total_correct += len(lab_chunks)
            
    p   = correct_preds / total_preds if correct_preds > 0 else 0
    r   = correct_preds / total_correct if correct_preds > 0 else 0
    f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0
    acc = np.mean(accs)

    return {"acc": 100*acc, "f1": 100*f1}

In [ ]:
def train(train, dev, epochs, epoch_no_improvement=4):
    for epoch in range(epochs):
        print(f"epoch {epoch} out of {epochs}")
        score = run_epoch(train, dev, epoch)
        lr *= 0.001#lr_decay # decay learning rate
        if score >= best_score:
            epo_no_improvement = 0
            save_session()
            best_score = score
            print("Best Score", best_score)
        else:
            epo_no_improvement += 1
            if epo_no_improvement > epoch_no_improvement:
                print(f"Early Stoping {epo_no_improvement} with no Improvement")
                break
                
        

In [ ]:
def build():
    """build model"""
    tf.reset_default_graph()
#     intialize_placeholders()
#     word_embedding()
#     logits_op()
#     pred_op()
#     loss_op()
    initialize_session()
    
    

In [ ]:
batch_size = 32
lr = 0.01

In [ ]:
train_data = [(i,j) for i, j in zip(x,y)]
dev_data = [(i,j) for i, j in zip(x_,y_)]

In [ ]:
# build()
train(train=train_data, dev=dev_data, epochs = 25, epoch_no_improvement=4)

In [ ]:
a = (x,y)

In [ ]:
for x,y in a:
    print(x)
    print(y)
    break